In [1]:
from Conference import Conference
from thConference import thConference
from wikicfputil import parseConference, parsethConf
import requests
from lxml import etree
import pymongo
import re
from datetime import datetime
import time
from operator import itemgetter, attrgetter

In [14]:
# 入库
def save(data, series=False, series_ids=None):
    if series:
        assert len(data) == len(series_ids)
    client = pymongo.MongoClient('127.0.0.1:27017')
    db = client.crawled_data
    saved = []
        
    if series:
        # 更新series库(wikicfp表)中的events
        for e,sid in zip(data,series_ids):
            record = db.wikicfp2.find_one({'series':sid})
            if record and e.year not in [th['year'] for th in record['event']]:
                record['event'].append(e.__dict__)
                infos = sorted(record['event'], key=lambda c: c['year'], reverse=True)
                saved.extend(infos)
                db.wikicfp2.update_one({'series':sid},{'$set':{'event':infos}})
            # else: print('thConference has saved or series id not found, sid:', sid)
        pass
    else:
        # 存放到 wikicfp_update 表中
        data = [d.__dict__ for d in data]
        
        # 去重
        for conf in data:
            if not db.wikicfp_update.find_one({'thabbr':conf['thabbr']}):
                db.wikicfp_update.insert_one(conf)
                saved.append(conf)
                
    print('Save update data: %d/%d:%s' % (len(saved), len(data), [s['thabbr'] for s in saved]))
    client.close()

def conftodict(conf):
    return {
            '_type': 'Conference',
            'name': conf.name,
            'abbr': conf.abbr,
            'series': conf.series,
            'link': conf.link,
            'main_link': conf.main_link,
            'past': conf.past,
            'future': conf.future,
            'event': [e.__dict__ for e in conf.event]
        }

def getAllAbbts():
    """
    获取所有的更新页面会议简称
    :return:
    """
    urls = [r'http://www.wikicfp.com/cfp/allcfp?page=' + str(i) for i in range(1, 11)]
    domin = r'http://www.wikicfp.com'
    thabbrs = []

    for url in urls:
        response = requests.get(url)
        page = etree.HTML(response.text)

        bgcolors = ['#f6f6f6', '#e6e6e6']
        for bgcolor in bgcolors:
            temp = page.xpath('//tr[@bgcolor="' + bgcolor + '"]/td/a/text()')
            thabbrs.extend(temp)
    print('Updated all thConference abbrs:', len(thabbrs))
    return thabbrs

# 抓取更新会议
1. 先抓取全部更新  
2. 有series的放到 series数组  没有的放到events  
3. 入库(未实现)

In [16]:
urls = [r'http://www.wikicfp.com/cfp/allcfp?page='+str(i) for i in range(1,11)]
domin = r'http://www.wikicfp.com'

# allcfp网页所有最新的届会议 并封装成conf对象
series_events = []
series_ids = []
events = []

for url in urls:
    response = requests.get(url)
    page = etree.HTML(response.text)

    print('抓取更新：', url[-1],'页')

    bgcolors = ['#f6f6f6', '#e6e6e6']
    for bgcolor in bgcolors:
        a_s = page.xpath('//tr[@bgcolor="' + bgcolor + '"]/td/a')
        for a in a_s:
            year = a.text.strip().split()[-1]
            year = re.findall('\d{1,4}', year)[0]
            event,series = parsethConf(thConference(link=domin + a.attrib['href'], thabbr=a.text, year=int(year)))
            if series: 
                series_events.append(event)
                series_ids.append(series)
            else: events.append(event)

# 入库保存
save(series_events, series=True, series_ids=series_ids)
save(events)
events.extend(series_events)

抓取更新： 1 页
抓取更新： 2 页
抓取更新： 3 页
抓取更新： 4 页
抓取更新： 5 页
抓取更新： 6 页
抓取更新： 7 页
抓取更新： 8 页
抓取更新： 9 页
抓取更新： 0 页
Save update data: 0/30:[]
Save update data: 170/170:['AxC 2019', 'AIAA 2018', 'Randomized Neural Networks 2019', 'ES 2019', 'ISEM 2019', 'FEAR 2019', 'MBE journal SCI 2019', 'CfP/OCS/ TI 2019', 'Cybersecurity-Track 2019', 'NSF-REU 2018', 'ICASSP 2019', 'IADIS IS 2019', 'ML 2019', 'BDESC 2018', 'FCT 2019', 'FABULOUS  2019', 'DISSECT 2019', 'Task Intelligence - WSDM 2019', 'TechAAL 2019', 'ET 2018', 'BlockDM 2019', 'ABS 2019', 'ICEE 2019', 'SOSR 2019', 'AI-DMR 2019', '5GWN 2019', 'HBPSC 2019', 'EuroPar 2018', 'ABB 2019', 'AI 2018', 'MUSICAL 2019', 'ICAR 2019', 'Metamaterials 2019', 'EG2019 Think Tan 2019', 'ICMT 2019', 'ICCNA 2018', 'EDUSREF 2019', 'META 2019', 'ICBDSDE 2018', 'EOLATAM 2019', 'SQUARE 2018', 'HAV 2019', 'SAC-CCNIV 2019', 'HAI 2018', 'AES 2019', 'ASPAI 2019', 'NFC 2018', 'CiDaS 2019', 'HMPMOS 2019', 'INTERSOL 2019', 'ACM-ICSENT 2018', 'GISD 2018', '2019 IEEE AITest 2019', 'I

# 更新抓取

In [ ]:

events_abbrs = [e.thabbr for e in events]
dict_new_series_ids = {}
dict_new_series_events = {}
dict_new_events = {}
while True:
    dict_new_events.clear()
    response = requests.get(urls[0])
    page = etree.HTML(response.text)

    print('Update checking：', datetime.now())

    bgcolors = ['#f6f6f6', '#e6e6e6']
    for bgcolor in bgcolors:
        a_s = page.xpath('//tr[@bgcolor="' + bgcolor + '"]/td/a')
        for a in a_s:
            year = a.text.strip().split()[-1]
            year = year[-4:]
            event,series = parsethConf(thConference(link=domin + a.attrib['href'], year=int(year), thabbr=a.text))
            if series:
                dict_new_series_ids[event.thabbr] = series
                dict_new_series_events[event.thabbr] = event
            else:
                dict_new_events[event.thabbr] = event
    
    # 非Series数据 
    new_events_abbrs = dict_new_events.keys()
    set_new_evnets_abbrs = set(new_events_abbrs) - set(events_abbrs)
    if(set_new_evnets_abbrs): 
        [print('New non-series record:', abbr) for abbr in set_new_evnets_abbrs]
        new_events = [dict_new_events[key] for key in set_new_evnets_abbrs]
        save(new_events)
    else: print('No non-series update data.')
    
    # Series数据 
    new_series_events_abbrs = dict_new_series_events.keys()
    set_new_series_evnets_abbrs = set(new_series_events_abbrs) - set(events_abbrs)
    if(set_new_series_evnets_abbrs): 
        [print('New series record:', abbr) for abbr in set_new_series_evnets_abbrs]
        new_events = [dict_new_series_events[key] for key in set_new_series_evnets_abbrs]
        save(new_events, series=True, series_ids=[dict_new_series_ids[k.thabbr] for k in new_events])
    else: print('No series update data.')
    
    # 更新缓存
    if set_new_evnets_abbrs or set_new_series_evnets_abbrs:
        events_abbrs = getAllAbbts()

    time.sleep(30*60)

Update checking： 2018-09-27 16:22:16.734499
No non-series update data.
No series update data.
Update checking： 2018-09-27 16:53:18.849621
No non-series update data.
No series update data.
Update checking： 2018-09-27 17:23:48.995885
No non-series update data.
No series update data.
Update checking： 2018-09-27 17:54:56.578805
No non-series update data.
No series update data.
Update checking： 2018-09-27 18:25:24.384673
New non-series record: SOCC 2019
New non-series record: ISKO-Maghreb 2018
New non-series record: IPRIA 2019
New non-series record: CRAS 2019
New non-series record: ECE 2019
New non-series record: PEFnet 2018
New non-series record: James Joyce Symposium 2019
New non-series record: IEEE SPW 2019
New non-series record: FinancialNews&Data-BigData2018
New non-series record: Co2hum 2018
New non-series record: IEEE WoWMoM 2019
Save update data: 11/11:['SOCC 2019', 'ISKO-Maghreb 2018', 'IPRIA 2019', 'CRAS 2019', 'ECE 2019', 'PEFnet 2018', 'James Joyce Symposium 2019', 'IEEE SPW 201